In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
articles=pd.read_csv('shared_articles.csv')
users=pd.read_csv('users_interactions.csv')

In [ ]:
articles.dropna(subset=['text'],inplace=True)
users.dropna(inplace=True)
users.columns = users.columns.str.strip()

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'BOOKMARK': 2.5,
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,
}


users['rating'] = users['eventType'].apply(lambda x: event_type_strength[x])
tdidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tdidf.fit_transform(articles['text'])

cosine_sim=cosine_similarity(tfidf_matrix,tfidf_matrix)

def get_recommendations(article_title,cosine_sim=cosine_sim):

  if article_title in articles['title'].values:
    idx = articles[articles['title'] == article_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]
    return articles['title'].iloc[article_indices]
  else:
    print(f"Article title '{article_title}' not found in the dataset.")
    return []

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
user_article_matrix = pd.pivot_table(users, values='rating', index='personId', columns='contentId', fill_value=0)
user_article_matrix_imputed = imputer.fit_transform(user_article_matrix)

sparse_matrix=csr_matrix(user_article_matrix)
print(user_article_matrix)
svd=TruncatedSVD(n_components=2)
latent_matrix=svd.fit_transform(sparse_matrix)

def get_collaborative_recommendation(personId):
  user_index = user_article_matrix.index.get_loc(personId)
  user_vector = latent_matrix[user_index]
  predicted_ratings = np.dot(latent_matrix, user_vector)
  recommended_indices = np.argsort(predicted_ratings)[-5:]
  return user_article_matrix.columns[recommended_indices]

contentId             -9222795471790223670  -9216926795620865886  \
personId                                                           
-9223121837663643404                   0.0                   0.0   
-9212075797126931087                   0.0                   0.0   
-9207251133131336884                   0.0                   1.0   
-9199575329909162940                   0.0                   0.0   
-9196668942822132778                   0.0                   0.0   
...                                    ...                   ...   
 9165571805999894845                   0.0                   0.0   
 9187866633451383747                   0.0                   0.0   
 9191849144618614467                   0.0                   0.0   
 9199170757466086545                   0.0                   0.0   
 9210530975708218054                   0.0                   0.0   

contentId             -9194572880052200111  -9192549002213406534  \
personId                                       

In [ ]:
def hybrid_recommendation(personId,article_title):
  content_recommendations=get_recommendations(article_title)
  collaborative_recommendations=get_collaborative_recommendation(personId)
  hybrid_recommendations=set(collaborative_recommendations).union(set(content_recommendations))
  return list(hybrid_recommendations)

In [ ]:
article_title='Why Decentralized Conglomerates Will Scale Better than Bitcoin - Interview with OpenLedger CEO - Bitcoin News'
personId=9210530975708218054


recommended_articles=hybrid_recommendation(personId,article_title)
print("recommended_articles: ")
for title in recommended_articles:
  print(title)

recommended_articles: 
Confused by blockchains? Revolution vs Evolution
Ethereum and Bitcoin Are Market Leaders But Not Competitors
Bitit Launches to Solve Bitcoin Biggest Issues - Accessibility and Fraud
Are you a bitcoin denialist? - Chris Skinner's blog
-969155230116728853
Growing Global Electricity Consumption Is Not Just Bitcoin's Fault
-2548446404469756145
-9137036168156595470
-538948733779286129
Governments vs Bitcoin: A Fight For Control
Blockchain CEO's Business Advice on Bitcoin Startups
French Senate Will Debate on Bitcoin Regulation
Our Goal is to Replace Your Need for a Bank, Says Blockchain Co-Founder and CEO Nicolas Cary
-5315378314308323942
Blockchain won't kill banks: Bitcoin pioneer
